In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
from ultralytics import solutions


SQUARE_SIZE = 512

# Classe que realiza o processamento de cada quadrado

class SquareProcessor:
    def __init__(self, square):
        self.square = square
        # Dicionário para armazenar as etapas de processamento
        self.stages = {'original': square.copy()}


    def adjust_exposure(self):
        """INCREASE INTENSITY"""
        self.square = cv2.convertScaleAbs(self.square, alpha=1.5, beta=0)
        self.square = cv2.addWeighted(self.square, 1.5, np.zeros(self.square.shape, self.square.dtype), 0, 0)
        self.stages['adjust_exposure'] = self.square.copy()

       # self.square = cv2.equalizeHist(self.square)

    def clahe(self):
        """Contrast Limited Adaptive Histogram Equalization"""
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
        self.square = clahe.apply(self.square)
        self.stages['clahe'] = self.square.copy()

    def count_objects(self):
        """Conta o número de objetos na imagem."""

        # Contar objetos na image
        model_path = '/media/williancaddd/CODES/fiotec/AETrampa/Labels Eggs.v9i.yolov11/runs/detect/train3/weights/best.pt'
        region=[(0, 0), (512, 0), (512, 512), (0, 512)]
        counter = solutions.ObjectCounter(model=model_path, region=region, show=False)
        image_contend = counter.count(np.array(cv2.cvtColor(self.square, cv2.COLOR_GRAY2BGR)))
        num_eggs = len(counter.boxes)
        print(f"Total de ovos na imagem: {num_eggs}")
        # Converter a imagem anotada de volta para o formato OpenCV
        #self.square = image_contend
        self.stages['final_contours'] = image_contend
    

        return len(counter.boxes)

    def process(self):
        """Executa todo o pipeline de processamento de imagem para o quadrado."""

        self.adjust_exposure()
        self.clahe()
        return self.count_objects()

    def plot_stages(self):
        """Exibe todas as etapas do processamento de imagem."""
        stages = self.stages
        n_stages = len(stages)

        lines = (n_stages + 2) // 3
        fig, axes = plt.subplots(lines, 3, figsize=(15, 5 * lines))
        axes = axes.flatten()

        for i, (stage_name, stage_img) in enumerate(stages.items()):
            axes[i].imshow(stage_img, cmap='gray')
            axes[i].set_title(stage_name)
            axes[i].axis('off')

        for j in range(i + 1, len(axes)):
            axes[j].axis('off')

        plt.tight_layout()
        plt.show()

# Classe responsável por processar a imagem inteira


class ImageProcessor:
    def __init__(self, image_path, square_size=SQUARE_SIZE, save_slices=False):
        self.image_path = image_path
        self.square_size = square_size
        self.contoured_squares = []
        self.save_slices = save_slices

        # Criar a pasta "slices" se save_slices for True
        if self.save_slices:
            os.makedirs('slices', exist_ok=True)

    def resize_to_fit_grid(self, image):
        """
        Redimensiona a imagem para que sua altura e largura sejam múltiplos do square_size.
        Isso é necessário para dividir a imagem em quadrados de tamanho square_size.

        Preenche com preto se necessário para ser uma imagem em 4k.
        """

        img_height, img_width = image.shape
        new_height = img_height + (self.square_size - img_height % self.square_size) % self.square_size
        new_width = img_width + (self.square_size - img_width % self.square_size) % self.square_size

        resized_image = np.zeros((new_height, new_width), dtype=np.uint8)
        resized_image[:img_height, :img_width] = image

        return resized_image


    def divide_and_process_image(self):
        """Divide a imagem redimensionada em quadrados e processa cada um."""
        image = cv2.imread(self.image_path, cv2.IMREAD_GRAYSCALE)
        resized_image = self.resize_to_fit_grid(image)
        img_height, img_width = resized_image.shape

        # Dividir a imagem em quadrados de tamanho square_size
        total_objects = 0
        for y in range(0, img_height, self.square_size):
            for x in range(0, img_width, self.square_size):
                # Extrair o quadrado da imagem redimensionada
                square = resized_image[y:y + self.square_size, x:x + self.square_size]

                # Processar o quadrado
                square_processor = SquareProcessor(square)
                objects_in_square = square_processor.process()

                total_objects += objects_in_square

                # Armazenar quadrado com contornos desenhados
                processed_square = square_processor.stages['final_contours']
                if len(processed_square.shape) == 3:  # Verifica se a imagem tem 3 canais
                    processed_square = cv2.cvtColor(processed_square, cv2.COLOR_BGR2GRAY)
                self.contoured_squares.append(processed_square)


                # Salvar o quadrado processado, se o save_slices for True
                if self.save_slices:
                    image_original_name = os.path.basename(self.image_path) # Nome da imagem original
                    slice_path = f"slices/slice_{y}_{x}_{image_original_name}.jpg"
                    cv2.imwrite(slice_path, square_processor.stages['final_contours'])

                # Exibir as etapas do processamento para cada quadrado
                square_processor.plot_stages()

        return total_objects, img_height, img_width

    def reconstruct_image(self, img_height, img_width):
        """Reconstrói a imagem a partir dos quadrados processados."""
        reconstructed_image = np.zeros((img_height, img_width), dtype=np.uint8)
        count = 0
        for y in range(0, img_height, self.square_size):
            for x in range(0, img_width, self.square_size):
                reconstructed_image[y:y + self.square_size, x:x + self.square_size] = self.contoured_squares[count]
                count += 1

        return reconstructed_image


# Execução do código
# Definir caminho para a imagem
image_path = '/media/williancaddd/CODES/fiotec/AETrampa/base-2/2_55_0.JPEG'

# Processar imagem e contar objetos
# Definir save_slices para True para salvar as fatias

batch_images = []
# read all image in the folder  /media/williancaddd/CODES/fiotec/AETrampa/base-1 /media/williancaddd/CODES/fiotec/AETrampa/base-2
save_slices=True
for filename in os.listdir('/media/williancaddd/CODES/fiotec/AETrampa/base-1'):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        batch_images.append( f"/media/williancaddd/CODES/fiotec/AETrampa/base-1/{filename}")

for filename in os.listdir('/media/williancaddd/CODES/fiotec/AETrampa/base-2'):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        batch_images.append( f"/media/williancaddd/CODES/fiotec/AETrampa/base-2/{filename}")

if save_slices:
    for image_path  in batch_images:
        print(f"Processando imagem: {image_path}")
        image_processor = ImageProcessor(image_path, save_slices=save_slices)
        total_objects, img_height, img_width = image_processor.divide_and_process_image()
        print(f"Total de objetos na imagem: {total_objects}")
else:
    image_processor = ImageProcessor(image_path, save_slices=save_slices)
    total_objects, img_height, img_width = image_processor.divide_and_process_image()
    print(f"Total de objetos na imagem: {total_objects}")

    # Reconstruir imagem processada
    reconstructed_image = image_processor.reconstruct_image(img_height, img_width)

    # Mostrar imagem original e reconstruída
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))
    original_image = cv2.imread(image_path)
    axes[0].imshow(original_image,)
    axes[0].set_title('Imagem Original')

    axes[1].imshow(reconstructed_image, cmap='gray')
    axes[1].set_title('Imagem Reconstruída')

    plt.show()
